[View in Colaboratory](https://colab.research.google.com/github/zacateras/nn-nbirds/blob/master/report_colab.ipynb)

In [15]:
% cd ~

# Remove the environment
! if [ -d "nn-nbirds" ]; then rm -rf "nn-nbirds"; fi
# ! pip freeze | xargs pip uninstall -y

# Build the environment 
! git clone https://github.com/zacateras/nn-nbirds.git
% cd ./nn-nbirds
! pip install -r requirements.txt

/content
Cloning into 'nn-nbirds'...
remote: Counting objects: 36, done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 36 (delta 12), reused 28 (delta 7), pack-reused 0
Unpacking objects: 100% (36/36), done.
/content/nn-nbirds
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [16]:
# Download the dataset
! ./download.sh

--2018-04-25 20:34:22--  https://www.dropbox.com/s/fi2g3zxsn0pdmn1/nbirds.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:601a:1::a27d:701
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/xqewx72E7wD5DrSgNjmqGuhxtWLCM15Sf8HqARHxi4KrPmuTTL7NkEAN5wgvEbXZ/file [following]
--2018-04-25 20:34:23--  https://dl.dropboxusercontent.com/content_link/xqewx72E7wD5DrSgNjmqGuhxtWLCM15Sf8HqARHxi4KrPmuTTL7NkEAN5wgvEbXZ/file
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.7.6, 2620:100:601a:6::a27d:706
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.7.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 623160299 (594M) [application/zip]
Saving to: ‘/content/nn-nbirds/data/nbirds.zip’

/content/nn-nbirds/ 100%[===================>] 594.29M  32.8MB/s    in 19s     

2

In [0]:
from preprocess import *

# preprocess and load txt ds metadata
ds_meta = build_ds_meta()

In [26]:
for ds_meta_item in ds_meta:
    print('Quantity of %s: %s' % (ds_meta_item, ds_meta[ds_meta_item].count()[0]))

Quantity of bounding_boxes: 48562
Quantity of classes: 1011
Quantity of hierarchy: 1010
Quantity of image_class_labels: 48562
Quantity of images: 48562
Quantity of photographers: 48562
Quantity of sizes: 48562


In [0]:
# BASE               SET_A
# + Bounding box  => SET_A_BB               (del)
# + Gabor filter  => SET_A_BB_GF            (del)
# + TVT splitting => SET_A_BB_GF_train
#                    SET_A_BB_GF_validation
#                    SET_A_BB_GF_test

apply_bounding_box('data/SET_A', 'data/SET_A_BB', ds_meta)
apply_gabor_filter('data/SET_A_BB', 'data/SET_A_BB_GF')
apply_tvt_split('data/SET_A_BB_GF')

In [5]:
import os

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


In [7]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
batch_size = 16

train_labels = os.listdir('data/SET_A_BB_GF_train')

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    'data/SET_A_BB_GF_train',
    target_size=(150, 150),
    batch_size=batch_size,
    classes=train_labels)

model = Sequential()
model.add(Flatten(input_shape=(150, 150, 3))) 
model.add(Dense(256))
model.add(Dense(512))
model.add(Dense(len(train_labels)))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50)

Found 2103 images belonging to 50 classes.
Epoch 1/50
125/125 [==============================] - 20s 161ms/step - loss: 10.0366 - acc: 0.0200
Epoch 2/50
125/125 [==============================] - 20s 163ms/step - loss: 10.3155 - acc: 0.0190
Epoch 3/50
125/125 [==============================] - 20s 164ms/step - loss: 10.5751 - acc: 0.0215
Epoch 4/50
 27/125 [=====>........................] - ETA: 16s - loss: 9.9018 - acc: 0.0185

125/125 [==============================] - 20s 162ms/step - loss: 10.2485 - acc: 0.0195
Epoch 5/50
125/125 [==============================] - 20s 161ms/step - loss: 10.2719 - acc: 0.0215
Epoch 6/50
101/125 [=======================>......] - ETA: 3s - loss: 10.6353 - acc: 0.0161